In [2]:
from pyairtable import Api
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('AIRTABLE_API_KEY')
base_id = os.getenv('AIRTABLE_BASE')

api = Api(api_key)

matches_table = api.table(base_id, 'matches')
people_table = api.table(base_id, 'people')

In [19]:
for record in matches_table.all():
    # get p1 insta
    p1 = people_table.get(record['fields']['p1'][0])
    p2 = people_table.get(record['fields']['p2'][0])

    p1_insta = p1['fields']['insta_handle'].replace('@', '')
    p2_insta = p2['fields']['insta_handle'].replace('@', '')

    p1_insta = "@" + p1_insta
    p2_insta = "@" + p2_insta

    email = record['fields']['email_body']
    email += f"\n\nP.S. According to our highly sophisticated love algorithm, you two have a {record['fields']['percent']*100}% compatibility score - looks like the odds are in favor of an epic partnership! 🚀💕"
    email += f"\n\nP.P.S. hit reply to this email with a cute note! emails are honestly the best love letters. after that check out each others insta (maybe?) {p1_insta} & {p2_insta} !"
    email += f"\n\n **P.P.P.S. If you're interested in meeting up, we might be hosting smth @ Ackerman on Valentines Day! No guarantees, but we'll send updates tmmrw. We'd love to see you there! 💌**"
    email += f"\n\nya'll should def organize a time to meet up!!! (and take pics!)"
    email += f"\n\n-hackin.la team"

    matches_table.update(record['id'], {'email_body': email})

In [22]:
import base64
from email.mime.text import MIMEText
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from requests import HTTPError

SCOPES = [
        "https://www.googleapis.com/auth/gmail.send"
    ]
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
creds = flow.run_local_server(port=0)

service = build('gmail', 'v1', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725730991270-om13lfcjhpoc2ptl1g4ajhoklgt0j6qf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53067%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=Fd45BlrD2wE1SN1r6BLlqwn2x91hmG&access_type=offline


In [40]:
import base64
import markdown
from tqdm import tqdm
import time


In [42]:
for record in tqdm(matches_table.all()):
    email = record['fields']['email_body']
    email = markdown.markdown(email)

    p1 = people_table.get(record['fields']['p1'][0])
    p2 = people_table.get(record['fields']['p2'][0])

    p1_email = p1['fields']['email']
    p2_email = p2['fields']['email']

    emails = f"{p1_email}, {p2_email}"

    message = MIMEText(email.replace('\n', '<br>'), 'html')
    message['to'] = emails
    message['subject'] = 'ai valentines day matches are out!!!'
    create_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}

    try:
        message = (service.users().messages().send(userId="me", body=create_message).execute())
        print(F'sent message to {message} Message Id: {message["id"]}')
    except HTTPError as error:
        print(F'An error occurred: {error}')
        message = None

    time.sleep(1) # avoid rate limit
    break

akshaykg925@gmail.com, swasinya@gmail.com
